In [67]:
import pandas as pd
%pip install folium 
import folium

Note: you may need to restart the kernel to use updated packages.


In [69]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [116]:
import requests
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
spacex_csv_file = io.BytesIO(response.content)
spacex_df=pd.read_csv(spacex_csv_file)

In [246]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df.head(10)

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [248]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [250]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='##0000FF', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [252]:

LC_40 = [28.56230,  -80.576820]
SLC_40=[28.563197,	-80.576820]
LC_39A=[28.573255,	-80.646895]
SLC4E= [34.632834,	-120.610745]
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=LC_40, zoom_start=10)
for coord, name in zip([LC_40, SLC_40, LC_39A, SLC4E],
                       ['LC 40', 'SLC 40', 'LC 39A', 'SLC4E']):
    folium.Circle(
        location=coord,
        radius=1000,
        color='#0000FF',  # only one '#'!
        fill=True
    ).add_child(folium.Popup(name)).add_to(site_map)

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    ).add_to(site_map)
marker = folium.map.Marker(
    LC_40,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,-10),
        html='<div style="font-size: 12; color:#FFA500;"><b>%s</b></div>' % 'LC-40',
        )
    ).add_to(site_map)
marker = folium.map.Marker(
    SLC_40,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(25,10),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'SLC-40',
        )
    ).add_to(site_map)
marker = folium.map.Marker(
    LC_39A,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#FFA500;"><b>%s</b></div>' % 'LC-39A',
        )
    ).add_to(site_map)
marker = folium.map.Marker(
    SLC4E,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'SLC4E',
        )
    )
success_rate_df=spacex_df.groupby('Launch Site')['class'].mean().reset_index()
success_rate_df.rename(columns={'class': 'Success Rate'}, inplace=True)
site_coords_df = spacex_df[['Launch Site', 'Lat', 'Long']]
site_stats_df = pd.merge(site_coords_df, success_rate_df, on='Launch Site')
for _, row in site_stats_df.iterrows():
    rate = round(row['Success Rate'] * 100, 2)
    label = f"<b>{row['Launch Site']}</b><br>Success Rate: {rate}%"
    
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(3, 7),
            html=f'<div style="font-size: 12pt; color: #2c3e50;">{label}</div>',
        )
    ).add_to(site_map)

site_map.add_child(circle)
site_map.add_child(marker)

In [254]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [256]:
marker_cluster = MarkerCluster().add_to(site_map)
for _, row in spacex_df.iterrows():
    color = 'green' if row['class'] == 1 else 'red'
    outcome = 'Success' if row['class'] == 1 else 'Failure'
    
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']}<br>{outcome}",
        icon=folium.Icon(color=color, icon='rocket', prefix='fa')
    ).add_to(marker_cluster)
site_map


In [258]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [260]:
from math import radians, cos, sin, asin, sqrt
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))

    distance = R * c
    return distance
launch_site_lat = 28.563197
launch_site_lon = -80.576820
coastline_lat = 28.56685
coastline_lon = -80.56866
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon,
                                        coastline_lat, coastline_lon)
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
   icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
       html=f'<div style="font-size: 10pt; color:#d35400;"><b>{distance_coastline:10.2f} KM</b></div>',
       )
   ).add_to(site_map)
site_map


In [261]:
lines =folium.PolyLine(
    locations= [[28.563197, -80.576820],[28.56685, -80.56866]],
    weight=1,
    color="Red"
)
site_map.add_child(lines)

In [263]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))

    distance = R * c
    return distance
launch_site_lat = 28.563197
launch_site_lon = -80.57794
railroad_lat = 28.5607
railroad_lon = -80.58747
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon,
                                        railroad_lat, railroad_lon)
distance_marker = folium.Marker(
    location=[railroad_lat, railroad_lon],
   icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
       html=f'<div style="font-size: 10pt; color:#d35400;"><b>{distance_coastline:10.2f} KM</b></div>',
       )
   ).add_to(site_map)
lines =folium.PolyLine(
    locations= [[28.562302, -80.577356],[28.5607, -80.5874]],
    weight=1,
    color="Red"
)
site_map.add_child(lines)
site_map

In [268]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))

    distance = R * c
    return distance
launch_site_lat = 28.573255
launch_site_lon = -80.646895
railroad_lat = 28.52533
railroad_lon = -80.6484
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon,
                                        railroad_lat, railroad_lon)
distance_marker = folium.Marker(
    location=[railroad_lat, railroad_lon],
   icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
       html=f'<div style="font-size: 10pt; color:#d35400;"><b>{distance_coastline:10.2f} KM</b></div>',
       )
   ).add_to(site_map)
lines =folium.PolyLine(
    locations= [[28.573255, -80.646895],[28.52533, -80.6484]],
    weight=1,
    color="Red"
)
site_map.add_child(lines)
site_map

In [272]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))

    distance = R * c
    return distance
launch_site_lat = 34.632834
launch_site_lon = -120.610745
railroad_lat = 34.63903
railroad_lon = -120.45792
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon,
                                        railroad_lat, railroad_lon)
distance_marker = folium.Marker(
    location=[railroad_lat, railroad_lon],
   icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
       html=f'<div style="font-size: 10pt; color:#d35400;"><b>{distance_coastline:10.2f} KM</b></div>',
       )
   ).add_to(site_map)
lines =folium.PolyLine(
    locations= [[34.632834, -120.610745],[34.63903, -120.45792]],
    weight=1,
    color="Red"
)
site_map.add_child(lines)
site_map